In [1]:
import sys
import os
import traceback

project_root_path = os.path.abspath('..')
sys.path.insert(0, project_root_path)

try:
    from src.data_management import load_data
    dataset_path = load_data()
except Exception as e:
    print(f"\n--- An error occurred ---")
    traceback.print_exc()


Current working directory: c:\Users\Infin\Documents\GoogleDriveSync\Obsidian\Künstliche Intelligenz Labor\Project Phase\X-Ray Pneumonia Detection\X-Ray Pneumonia Detection\Notebooks
Adding to sys.path: c:\Users\Infin\Documents\GoogleDriveSync\Obsidian\Künstliche Intelligenz Labor\Project Phase\X-Ray Pneumonia Detection\X-Ray Pneumonia Detection
sys.path starts with: ['c:\\Users\\Infin\\Documents\\GoogleDriveSync\\Obsidian\\Künstliche Intelligenz Labor\\Project Phase\\X-Ray Pneumonia Detection\\X-Ray Pneumonia Detection', 'C:\\Users\\Infin\\miniconda3\\python312.zip', 'C:\\Users\\Infin\\miniconda3\\DLLs']
Successfully imported load_data from src.data_management.


c:\Users\Infin\Documents\GoogleDriveSync\Obsidian\Künstliche Intelligenz Labor\Project Phase\X-Ray Pneumonia Detection\X-Ray Pneumonia Detection\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 2.29G/2.29G [07:19<00:00, 5.61MB/s]

Extracting files...


Dataset download complete. Path to dataset files: C:\Users\Infin\Documents\GoogleDriveSync\Obsidian\Künstliche Intelligenz Labor\Project Phase\X-Ray Pneumonia Detection\X-Ray Pneumonia Detection\data\raw\datasets\paultimothymooney\chest-xray-pneumonia\versions\2

--- Success! ---
The returned path is: C:\Users\Infin\Documents\GoogleDriveSync\Obsidian\Künstliche Intelligenz Labor\Project Phase\X-Ray Pneumonia Detection\X-Ray Pneumonia Detection\data\raw\datasets\paultimothymooney\chest-xray-pneumonia\versions\2
Verification: The returned path exists.
